Übung: Volume und Claims
------------------------

Zuerst brauchen wir ein `PersistentVolume` worauf die `Claims` zusteuern.

Im nachfolgenden Beispiel wird der `hostPath` als Volume zur Verfügung gestellt:

    kind: PersistentVolume
    apiVersion: v1
    metadata:
      name: data-volume
      labels:
        type: local
    spec:
      storageClassName: manual
      capacity:
        storage: 10Gi
      accessModes:
        - ReadWriteMany
      hostPath:
        path: "/data"


Anschliessend folgen die `Claims`:

    kind: PersistentVolumeClaim
    apiVersion: v1
    metadata:
      name: data-claim
    spec:
      storageClassName: manual
      accessModes:
        - ReadWriteMany
      resources:
        requests:
          storage: 10Gi

Die Pod selber zeigen auf die `Claims`:

      volumes:
      - name: web-storage
        persistentVolumeClaim:
        claimName: data-claim    

Der Pod, muss zuerst aber noch, mittels `volumeMounts` seine Verzeichnisse freigeben:

    volumeMounts:
    - mountPath: "/usr/local/apache2/htdocs"
      subPath: htdocs       
      name: "web-storage"

Das komplette Beispiel eines Web Server welche seine Dateien auf dem Host ablegt sieht wie folgt aus:


In [ ]:
! cat 09-5-Volume/apache.yaml 

In [ ]:
! kubectl apply -f 09-5-Volume/apache.yaml

Um mehr als eine leere Seite anzuzeigen, muss jetzt im Verzeichnis `/data/htdocs` auf dem **in der VM** eine Datei `index.html` angelegt werden. 

Der Inhalt kann wir folgt erstellt werden:

    cat <<%EOF% >/data/htdocs/index.html
        <html>
            <body>
                <h1>Hallo Host</h1>
            </body>
        </html>
    %EOF%
    ls -l /data/htdocs


In [ ]:
! kubectl get service/apache

Da wir keinen LoadBalancer haben müssen wir mit einem kleinen Shellscript selber die IP des Clusters und der gemappte Port (port-based-routing) als URL aufbereiten

Wird dieser URL in einem neuen Tab geöffnet, wird `Hallo Host` angezeigt.

In [ ]:
! echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service apache -o=jsonpath='{ .spec.ports[0].nodePort }')

- - -

Aufräumen. 

Die Datei `lernkube/data/htdocs/index.html` bleibt erhalten, weil auf `PersistentVolume` gespeichert.

In [ ]:
! kubectl delete -f 09-5-Volume/apache.yaml